In [3]:
pd.options.display.max_rows = 10

In [2]:
import pandas as pd
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile
from arcgis.features import FeatureLayer
import numpy as np


https://www.datos.gov.co/Estad-sticas-Nacionales/Entradas-de-extranjeros-a-Colombia/96sh-4v8d


In [4]:
df = pd.read_csv("Entradas_de_extranjeros_a_Colombia.csv",dtype={'Nueva columna georreferenciada':'str'})
df=df.dropna(axis=0, subset=['Nueva columna georreferenciada'])
df=df[df['Nacionalidad'] == 'Venezuela']
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,Nueva columna georreferenciada
907,2012,Enero,Venezuela,297,339,NaN,636,POINT (0.81488 -77356)
908,2012,Enero,Venezuela,7,12,NaN,19,POINT (10406342 -75527794)
909,2012,Enero,Venezuela,76,133,NaN,209,POINT (10446304 -75516486)
910,2012,Enero,Venezuela,55,70,NaN,125,POINT (10886686 -74776479)
911,2012,Enero,Venezuela,0,4,NaN,4,POINT (11117206 -74232675)
...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,POINT (7128047 -73181317)
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,POINT (7819 -72453013)
67655,2019,Junio,Venezuela,12,28,NaN,40,POINT (7927233 -72508188)
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,POINT (836068 -72409396)


In [5]:
lonlat=(df
           .apply(lambda row: row['Nueva columna georreferenciada'],axis=1)
           .apply(lambda word: word.replace('POINT (',''))
           .apply(lambda body: list(body))
           .apply(lambda letters: list(map(lambda letter:letter.replace(")",''),letters)))
           .apply(lambda letters: ''.join(letters))
              )            

df['lat long']=lonlat
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,Nueva columna georreferenciada,lat long
907,2012,Enero,Venezuela,297,339,NaN,636,POINT (0.81488 -77356),0.81488 -77356
908,2012,Enero,Venezuela,7,12,NaN,19,POINT (10406342 -75527794),10406342 -75527794
909,2012,Enero,Venezuela,76,133,NaN,209,POINT (10446304 -75516486),10446304 -75516486
910,2012,Enero,Venezuela,55,70,NaN,125,POINT (10886686 -74776479),10886686 -74776479
911,2012,Enero,Venezuela,0,4,NaN,4,POINT (11117206 -74232675),11117206 -74232675
...,...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,POINT (7128047 -73181317),7128047 -73181317
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,POINT (7819 -72453013),7819 -72453013
67655,2019,Junio,Venezuela,12,28,NaN,40,POINT (7927233 -72508188),7927233 -72508188
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,POINT (836068 -72409396),836068 -72409396


In [169]:
df.iloc[18]

Año                                                     2012
Mes                                                  Febrero
Nacionalidad                                       Venezuela
Femenino                                                   0
Masculino                                                  3
                                             ...            
Total                                                      3
Nueva columna georreferenciada    POINT (-4191162 -69941498)
lonlat                                    -4191162 -69941498
latt                                                     NaN
lonn                                                     NaN
Name: 1883, Length: 11, dtype: object

In [6]:
latlon_df = df['lat long'].apply(lambda x: pd.Series(x.split(' ')))
df['latt']=latlon_df[0]
df['lonn']=latlon_df[1]
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,Nueva columna georreferenciada,lat long,latt,lonn
907,2012,Enero,Venezuela,297,339,NaN,636,POINT (0.81488 -77356),0.81488 -77356,0.81488,-77356
908,2012,Enero,Venezuela,7,12,NaN,19,POINT (10406342 -75527794),10406342 -75527794,10406342,-75527794
909,2012,Enero,Venezuela,76,133,NaN,209,POINT (10446304 -75516486),10446304 -75516486,10446304,-75516486
910,2012,Enero,Venezuela,55,70,NaN,125,POINT (10886686 -74776479),10886686 -74776479,10886686,-74776479
911,2012,Enero,Venezuela,0,4,NaN,4,POINT (11117206 -74232675),11117206 -74232675,11117206,-74232675
...,...,...,...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,POINT (7128047 -73181317),7128047 -73181317,7128047,-73181317
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,POINT (7819 -72453013),7819 -72453013,7819,-72453013
67655,2019,Junio,Venezuela,12,28,NaN,40,POINT (7927233 -72508188),7927233 -72508188,7927233,-72508188
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,POINT (836068 -72409396),836068 -72409396,836068,-72409396


In [32]:
df=df.drop(['lat long', 'Nueva columna georreferenciada'], axis=1)
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,latt,lonn,long
907,2012,Enero,Venezuela,297,339,NaN,636,0.81488,-77356,-77.356
908,2012,Enero,Venezuela,7,12,NaN,19,10406342,-75527794,-75.527794
909,2012,Enero,Venezuela,76,133,NaN,209,10446304,-75516486,-75.516486
910,2012,Enero,Venezuela,55,70,NaN,125,10886686,-74776479,-74.776479
911,2012,Enero,Venezuela,0,4,NaN,4,11117206,-74232675,-74.232675
...,...,...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,7128047,-73181317,-73.181317
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,7819,-72453013,-72.453013
67655,2019,Junio,Venezuela,12,28,NaN,40,7927233,-72508188,-72.508188
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,836068,-72409396,-72.409396


In [201]:
df = pd.read_csv("Libro1.csv", sep=';',dtype={'columna':'str','georreferenciada':'str'})
df=df.dropna(axis=0, subset=['georreferenciada'])
df=df.dropna(axis=0, subset=['columna'])
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,Nueva,columna,georreferenciada
0,2012,Enero,Venezuela,297.0,339.0,NaN,636.0,POINT,0.81488,-77356
1,2012,Enero,Venezuela,7.0,12.0,NaN,19.0,POINT,10406342,-75527794
2,2012,Enero,Venezuela,76.0,133.0,NaN,209.0,POINT,10446304,-75516486
3,2012,Enero,Venezuela,55.0,70.0,NaN,125.0,POINT,10886686,-74776479
4,2012,Enero,Venezuela,0.0,4.0,NaN,4.0,POINT,11117206,-74232675
...,...,...,...,...,...,...,...,...,...,...
1729,2019,Junio,Venezuela,9.0,13.0,NaN,22.0,POINT,7128047,-73181317
1730,2019,Junio,Venezuela,29976.0,26826.0,NaN,56802.0,POINT,7819,-72453013
1731,2019,Junio,Venezuela,12.0,28.0,NaN,40.0,POINT,7927233,-72508188
1732,2019,Junio,Venezuela,1904.0,1597.0,NaN,3501.0,POINT,836068,-72409396


In [200]:
df=df.dropna(axis=0, subset=['latt'])
df=df.dropna(axis=0, subset=['lonn'])
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,latt,lonn,long
907,2012,Enero,Venezuela,297,339,NaN,636,0.81488,-77356,0
908,2012,Enero,Venezuela,7,12,NaN,19,10406342,-75527794,0
909,2012,Enero,Venezuela,76,133,NaN,209,10446304,-75516486,0
910,2012,Enero,Venezuela,55,70,NaN,125,10886686,-74776479,0
911,2012,Enero,Venezuela,0,4,NaN,4,11117206,-74232675,0
...,...,...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,7128047,-73181317,0
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,7819,-72453013,0
67655,2019,Junio,Venezuela,12,28,NaN,40,7927233,-72508188,0
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,836068,-72409396,0


In [23]:
long=df['lonn']
df['long']=0
for i in range(1734):
     df['long'].iloc[i]=(long.iloc[i])[0:3]+'.'+(long.iloc[i])[3:]
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,latt,lonn,long
907,2012,Enero,Venezuela,297,339,NaN,636,0.81488,-77356,-77.356
908,2012,Enero,Venezuela,7,12,NaN,19,10406342,-75527794,-75.527794
909,2012,Enero,Venezuela,76,133,NaN,209,10446304,-75516486,-75.516486
910,2012,Enero,Venezuela,55,70,NaN,125,10886686,-74776479,-74.776479
911,2012,Enero,Venezuela,0,4,NaN,4,11117206,-74232675,-74.232675
...,...,...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,7128047,-73181317,-73.181317
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,7819,-72453013,-72.453013
67655,2019,Junio,Venezuela,12,28,NaN,40,7927233,-72508188,-72.508188
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,836068,-72409396,-72.409396


In [31]:
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,latt,lonn,long,lati,lat
907,2012,Enero,Venezuela,297,339,NaN,636,0.81488,-77356,-77.356,0,0
908,2012,Enero,Venezuela,7,12,NaN,19,10406342,-75527794,-75.527794,0,0
909,2012,Enero,Venezuela,76,133,NaN,209,10446304,-75516486,-75.516486,0,0
910,2012,Enero,Venezuela,55,70,NaN,125,10886686,-74776479,-74.776479,0,0
911,2012,Enero,Venezuela,0,4,NaN,4,11117206,-74232675,-74.232675,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,7128047,-73181317,-73.181317,0,0
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,7819,-72453013,-72.453013,0,0
67655,2019,Junio,Venezuela,12,28,NaN,40,7927233,-72508188,-72.508188,0,0
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,836068,-72409396,-72.409396,0,0


In [36]:
lat=df['latt']
df['lat']=0
for i in range(1734):
    if (lat.iloc[i])[0]=='1':
        df['lat'].iloc[i]=(lat.iloc[i])[0:2]+'.'+(lat.iloc[i])[2:]
    elif (lat.iloc[i])[0]=='-':
        df['lat'].iloc[i]=(lat.iloc[i])[1:2]+'.'+(lat.iloc[i])[2:]
    elif (lat.iloc[i])[0]=='0':
        df['lat'].iloc[i]=(lat.iloc[i])[0:1]+'.'+(lat.iloc[i])[2:]
    else:
        df['lat'].iloc[i]=(lat.iloc[i])[0:1]+'.'+(lat.iloc[i])[1:]       


In [39]:
df= df.drop(columns=['latt', 'lonn'], axis=1)
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,long,lat
907,2012,Enero,Venezuela,297,339,NaN,636,-77.356,0.81488
908,2012,Enero,Venezuela,7,12,NaN,19,-75.527794,10.406342
909,2012,Enero,Venezuela,76,133,NaN,209,-75.516486,10.446304
910,2012,Enero,Venezuela,55,70,NaN,125,-74.776479,10.886686
911,2012,Enero,Venezuela,0,4,NaN,4,-74.232675,11.117206
...,...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,-73.181317,7.128047
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,-72.453013,7.819
67655,2019,Junio,Venezuela,12,28,NaN,40,-72.508188,7.927233
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,-72.409396,8.36068


In [40]:
df[['long','lat']] = df[['long','lat']].apply(pd.to_numeric)
df

,Año,Mes,Nacionalidad,Femenino,Masculino,Indefinido,Total,long,lat
907,2012,Enero,Venezuela,297,339,NaN,636,-77.356000,0.814880
908,2012,Enero,Venezuela,7,12,NaN,19,-75.527794,10.406342
909,2012,Enero,Venezuela,76,133,NaN,209,-75.516486,10.446304
910,2012,Enero,Venezuela,55,70,NaN,125,-74.776479,10.886686
911,2012,Enero,Venezuela,0,4,NaN,4,-74.232675,11.117206
...,...,...,...,...,...,...,...,...,...
67653,2019,Junio,Venezuela,9,13,NaN,22,-73.181317,7.128047
67654,2019,Junio,Venezuela,29976,26826,NaN,56802,-72.453013,7.819000
67655,2019,Junio,Venezuela,12,28,NaN,40,-72.508188,7.927233
67656,2019,Junio,Venezuela,1904,1597,NaN,3501,-72.409396,8.360680


In [41]:
df.dtypes

Año               int64
Mes              object
Nacionalidad     object
Femenino          int64
Masculino         int64
Indefinido      float64
Total             int64
long            float64
lat             float64
dtype: object

In [42]:
df.to_csv('venezuelafinal2.csv', sep=',', encoding='utf-8')

In [248]:
df2 = pd.DataFrame(pd.read_csv("venezuelafinal.csv"))
df2.dtypes

Unnamed: 0        int64
Año               int64
Mes              object
Nacionalidad     object
Femenino        float64
Masculino       float64
Indefinido      float64
Total           float64
long            float64
lat             float64
dtype: object

In [46]:
gis = GIS("https://www.arcgis.com", username="rcmartinb", password="rcmbcpmb1526")

In [4]:
venezuela_properties = {
    'title': 'Venezolanos',
    'tags': ['migracion1','migracion2'],
    'type': 'CSV'
}
venezuela_file_location = 'venezuelafinal.csv'
venezuela_csv = gis.content.add(venezuela_properties, data=venezuela_file_location)
venezuela_csv

<Item title:"Venezolanos" type:CSV owner:rcmartinb>

In [5]:
venezuela_feature_layer_item = venezuela_csv.publish()

# Then visualize the new feature layer

venezuela_feature_layer_item

<Item title:"Venezolanos" type:Feature Layer Collection owner:rcmartinb>

In [9]:
m = gis.map()

# wait for the map to load, and then add the following in a new cell
m.center = [4, -74]           # `[latitude, longitude]`
m.zoom = 11
m

MapView(layout=Layout(height='400px', width='100%'), zoom=11.0)

In [10]:
query = 'title: "Venezolanos*" AND type: "Feature Service"'
search_results = gis.content.search(query=query, max_items=10)
search_results

[<Item title:"Venezolanos" type:Feature Layer Collection owner:rcmartinb>,
 <Item title:"Informacion estadistica habitantes venezolanos en la localidad de fontibon_fieldworker" type:Feature Layer Collection owner:jsacevedos_UDFJC>]

In [11]:
csv_data = search_results[0]
csv_data.url

'https://services.arcgis.com/deQSb0Gn7gDPf3uV/arcgis/rest/services/Venezolanos/FeatureServer'

In [12]:
feature_layers = csv_data.layers
trailheads_layer = feature_layers[0]
trailheads_layer.properties.name

'Venezolanos'